# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 11:00AM,241083,20,8217559,"ACI Healthcare USA, Inc.",Released
1,Jul 5 2022 11:00AM,241083,20,8217560,"ACI Healthcare USA, Inc.",Released
2,Jul 5 2022 11:00AM,241083,20,8217582,"ACI Healthcare USA, Inc.",Released
3,Jul 5 2022 11:00AM,241083,20,8217583,"ACI Healthcare USA, Inc.",Released
4,Jul 5 2022 11:00AM,241083,20,8217597,"ACI Healthcare USA, Inc.",Released
5,Jul 5 2022 11:00AM,241083,20,8217608,"ACI Healthcare USA, Inc.",Released
6,Jul 5 2022 11:00AM,241083,20,8217610,"ACI Healthcare USA, Inc.",Released
7,Jul 5 2022 11:00AM,241083,20,8217611,"ACI Healthcare USA, Inc.",Released
8,Jul 5 2022 11:00AM,241083,20,8217614,"ACI Healthcare USA, Inc.",Released
9,Jul 5 2022 11:00AM,241083,20,8217615,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
60,241079,Released,9
61,241080,Released,42
62,241081,Released,8
63,241082,Released,63
64,241083,Released,23


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241079,NaN,9.0
241080,NaN,42.0
241081,NaN,8.0
241082,NaN,63.0
241083,NaN,23.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241004,53.0,6.0
241005,0.0,54.0
241006,0.0,55.0
241009,0.0,60.0
241010,0.0,60.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241004,53,6
241005,0,54
241006,0,55
241009,0,60
241010,0,60


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241004,53,6
1,241005,0,54
2,241006,0,55
3,241009,0,60
4,241010,0,60


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241004,53,6
1,241005,,54
2,241006,,55
3,241009,,60
4,241010,,60


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc."
23,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation
86,Jul 5 2022 10:51AM,241080,10,ISDIN Corporation
128,Jul 5 2022 10:50AM,241081,10,ISDIN Corporation
136,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.
165,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation
174,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation
176,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC
185,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC"
186,Jul 5 2022 10:30AM,241075,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc.",,23
1,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation,,63
2,Jul 5 2022 10:51AM,241080,10,ISDIN Corporation,,42
3,Jul 5 2022 10:50AM,241081,10,ISDIN Corporation,,8
4,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,,29
5,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation,,9
6,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation,,2
7,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC,,9
8,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC",,1
9,Jul 5 2022 10:30AM,241075,10,Emerginnova,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc.",23,
1,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation,63,
2,Jul 5 2022 10:51AM,241080,10,ISDIN Corporation,42,
3,Jul 5 2022 10:50AM,241081,10,ISDIN Corporation,8,
4,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,29,
5,Jul 5 2022 10:46AM,241079,10,ISDIN Corporation,9,
6,Jul 5 2022 10:43AM,241078,10,ISDIN Corporation,2,
7,Jul 5 2022 10:32AM,241076,10,MedStone Pharma LLC,9,
8,Jul 5 2022 10:32AM,241077,10,"Nextsource Biotechnology, LLC",1,
9,Jul 5 2022 10:30AM,241075,10,Emerginnova,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc.",23,
1,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation,63,
2,Jul 5 2022 10:51AM,241080,10,ISDIN Corporation,42,
3,Jul 5 2022 10:50AM,241081,10,ISDIN Corporation,8,
4,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,29,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc.",23.0,NaN
1,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation,63.0,NaN
2,Jul 5 2022 10:51AM,241080,10,ISDIN Corporation,42.0,NaN
3,Jul 5 2022 10:50AM,241081,10,ISDIN Corporation,8.0,NaN
4,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,29.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc.",23.0,0.0
1,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation,63.0,0.0
2,Jul 5 2022 10:51AM,241080,10,ISDIN Corporation,42.0,0.0
3,Jul 5 2022 10:50AM,241081,10,ISDIN Corporation,8.0,0.0
4,Jul 5 2022 10:49AM,241062,20,Alumier Labs Inc.,29.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,11811119,1721.0,132.0
12,723140,71.0,11.0
15,1446390,103.0,0.0
19,241057,1.0,0.0
20,482145,52.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,11811119,1721.0,132.0
1,12,723140,71.0,11.0
2,15,1446390,103.0,0.0
3,19,241057,1.0,0.0
4,20,482145,52.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1721.0,132.0
1,12,71.0,11.0
2,15,103.0,0.0
3,19,1.0,0.0
4,20,52.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1721.0
1,12,Released,71.0
2,15,Released,103.0
3,19,Released,1.0
4,20,Released,52.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Executing,132.0,11.0,0.0,0.0,0.0
Released,1721.0,71.0,103.0,1.0,52.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Executing,132.0,11.0,0.0,0.0,0.0
1,Released,1721.0,71.0,103.0,1.0,52.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Executing,132.0,11.0,0.0,0.0,0.0
1,Released,1721.0,71.0,103.0,1.0,52.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()